In [33]:
import numpy as np
from PIL import Image

def destuff_bits(binary_string):
    """
    Removing '1' inserted after every 5 consecutive '0's in the binary string.

    Args:
        binary_string (str): Binary string to be destuffed.

    Returns:
        str: Binary string after destuffing.
    """
    result = ''
    count = 0

    i = 0
    while i < len(binary_string):
        bit = binary_string[i]
        result += bit
        if bit == '0':
            count += 1
            if count == 5:
                # Skip the next bit if it is '1'
                if i + 1 < len(binary_string) and binary_string[i + 1] == '1':
                    i += 1
                count = 0
        else:
            count = 0
        i += 1

    return result

# Constants
PIXEL_COLOR_MAP = {
    (255, 255, 0): '4',  # Yellow
    (255, 0, 0): '3',    # Red
    (0, 255, 0): '2',    # Green
    (255, 255, 255): '1',# White
    (0, 0, 0): '0'       # Black
}
BUS_RATE = 512000  # 512 kbps

# Main function
def process_image(image_path):
    image = Image.open(image_path)
    pixels = np.array(image)
    
    rows, cols, _ = pixels.shape
    frames = []
    current_frame = []
    idle_time = 0
    timestamp = 0

    in_frame = False
    after_idle = False

    for row in range(rows):
        for col in range(cols):
            pixel = tuple(pixels[row, col])
            if pixel in PIXEL_COLOR_MAP:
                value = PIXEL_COLOR_MAP[pixel]
                if value in '01':
                    if after_idle:
                        frames.append((current_frame, idle_time))
                        current_frame = []
                        idle_time = 0
                        after_idle = False
                    current_frame.append(value)
                    in_frame = True
                elif value == '2':
                    in_frame = False
                    idle_time += 1
                    after_idle = True
                elif value in '34':
                    in_frame = False
                    after_idle = True

    # Append the last frame if there is any
    if current_frame:
        frames.append((current_frame, idle_time))

    dataset = []

    for frame, idle_time in frames:
        binary_string = ''.join(frame)
        binary_string = destuff_bits(binary_string)
        can_id = hex(int(binary_string[1:12], 2))[2:].zfill(3)
        dlc = int(binary_string[16:20], 2)
        data_bytes = [hex(int(binary_string[20 + i*8:28 + i*8], 2))[2:].zfill(2) for i in range(dlc)]
        
        dataset.append({
            'timestamp': round(timestamp, 6),
            'can_id': can_id,
            'dlc': dlc,
            'data': data_bytes
        })
        
        frame_length = len(frame)
        timestamp += (frame_length / BUS_RATE )+ (idle_time / BUS_RATE)

    return dataset

def save_to_txt(dataset, file_path):
    with open(file_path, 'w') as file:
        for data in dataset:
            data_bytes_str = ','.join(data['data'])
            file.write(f"{data['timestamp']:.6f},{data['can_id']},{data['dlc']},{data_bytes_str}\n")

# Example usage
image_path = "D:\\IIT-D\\Sem-4\\JCD893-M.Tech Major Project Part-2\\Anchor_Frame\\Generated_Images\\new_stuff\\test\\attack\\image_43.png"
dataset = process_image(image_path)
save_to_txt(dataset, "output.txt")
